# Analyse des Données Brutes HDFS et Séparation en Entraînement/Test

Ce notebook examine les données brutes du fichier de logs HDFS (`HDFS.log`) et les divise en deux ensembles : un ensemble d'entraînement et un ensemble de test. L'objectif est de créer une base de données pour entraîner un modèle de détection d'anomalies dans les logs, en s'appuyant sur l’outil de parsing `Drain`. Ce dernier génère des "templates" (modèles) de messages pour les logs structurés, ce qui simplifie l'identification de motifs d'anomalies.

Les données HDFS utilisées proviennent de la collection [Loghub](https://github.com/logpai/loghub), une grande base de données de logs systèmes. L'ensemble de données utilisé contient également des annotations pour chaque bloc indiquant s'il est normal ou anormal, ce qui facilite la validation du modèle.


## Importation des Bibliothèques

In [2]:
## Importation des bibliothèques essentielles pour l'analyse et la manipulation des données
import pandas as pd
import numpy as np
import logging
from itertools import islice

## Exploration Préliminaire des Données

Dans cette section, nous allons examiner les données brutes dans le fichier `HDFS.log`. Cela inclut le comptage des lignes pour comprendre la taille des données et l'observation des premières lignes pour obtenir un aperçu de la structure des logs.


In [6]:
# Compter le nombre de lignes dans le fichier HDFS_2k.log
with open('DataSet_HDFS/HDFS.log', "r") as file:
    totaln = sum(1 for line in file)
print(f'Le fichier HDFS.log contient un total de {totaln} lignes')

Le fichier HDFS.log contient un total de 11175629 lignes


In [8]:
# Afficher un aperçu rapide des 200 premières lignes du fichier
with open('DataSet_HDFS/HDFS.log', "r") as file:
    data = list(islice(file, 200))

In [9]:
# Afficher les premières lignes sous forme de DataFrame
df = pd.DataFrame(data, columns=['Log Entries'])
df.head()

,Log Entries
0,081109 203518 143 INFO dfs.DataNode$DataXceive...
1,081109 203518 35 INFO dfs.FSNamesystem: BLOCK*...
2,081109 203519 143 INFO dfs.DataNode$DataXceive...
3,081109 203519 145 INFO dfs.DataNode$DataXceive...
4,081109 203519 145 INFO dfs.DataNode$PacketResp...


islice(file, 200) : Utilise islice pour lire seulement les 200 premières lignes du fichier. islice (de la bibliothèque itertools) est une fonction pratique pour extraire une plage d'éléments sans boucle explicite.
Conversion en DataFrame : On peut directement transformer les lignes en DataFrame avec une seule colonne appelée "Log Entries".

In [10]:
df.iloc[50:100].values

array([['081109 203523 33 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.250.11.100:50010 is added to blk_-3544583377289625738 size 11971\n'],
       ['081109 203523 33 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.197.226:50010 is added to blk_-3544583377289625738 size 11971\n'],
       ['081109 203524 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-9073992586687739851 src: /10.250.14.196:39576 dest: /10.250.14.196:50010\n'],
       ['081109 203524 144 INFO dfs.DataNode$DataXceiver: Receiving block blk_-9073992586687739851 src: /10.250.7.244:39735 dest: /10.250.7.244:50010\n'],
       ['081109 203524 145 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_-9073992586687739851 terminating\n'],
       ['081109 203524 145 INFO dfs.DataNode$PacketResponder: Received block blk_-9073992586687739851 of size 11977 from /10.250.19.102\n'],
       ['081109 203524 146 INFO dfs.DataNode$PacketResponder: Pack

## Choix du Parseur Drain pour Structurer les Logs Non Structurés

Les données que nous utilisons sont des fichiers de logs non structurés. 
En consultant :
- **ICSE'19** : [Tools and Benchmarks for Automated Log Parsing](https://arxiv.org/pdf/1811.03509.pdf), par Jieming Zhu, Shilin He, Jinyang Liu, et al.

On constate que le parseur Drain a été identifié comme l'un des plus précis pour cette tâche. 

Drain est une méthode de parsing de logs qui utilise une structure d’arbre pour créer des modèles fixes (ou "templates") de messages. Cette méthode de parsing est intégrée dans l’outil Logparser, qui propose plusieurs méthodes de parsing de logs automatisées.

L’outil Logparser, qui inclut l’implémentation de Drain, est disponible dans le [Logparser toolkit](https://github.com/logpai/logparser). Il est cependant à noter que Drain est initialement implémenté en Python 2.7, et l’exécution doit se faire dans cet environnement. Dans notre cas, le script `project_parser.py` utilise Drain pour parser les fichiers `HDFS_2k.log` et `HDFS_2k_train.log`.


## Séparation des Données en Entraînement et Test

Pour évaluer la performance de notre modèle de détection d'anomalies, nous allons diviser les données en deux ensembles : un ensemble d'entraînement (80 % des données) et un ensemble de test (20 % des données). En gardant l'ordre des logs, nous utilisons les dernières données pour le test, simulant ainsi des "nouvelles anomalies" à découvrir.

In [11]:
# Calcul de l'indice où se termine l'ensemble d'entraînement (80 % des données)
train_idx = int(totaln * 0.8)
print("Indice de séparation entre entraînement et test :", train_idx)


Indice de séparation entre entraînement et test : 8940503


### Création du Fichier d’Entraînement HDFS_2k_train.log

In [12]:
# Extraction des 80 % premières lignes pour créer un fichier de logs d'entraînement.
# Ces données seront utilisées pour générer les modèles de templates dans l’ensemble d’entraînement.
with open('DataSet_HDFS/HDFS.log', "r") as file:
    train_data = list(islice(file, train_idx))

In [13]:
# write to the training file `HDFS_train.log`
with open('DataSet_HDFS/HDFS_train.log', 'x') as file:
    for i in train_data:
        file.write(i)

FileExistsError: [Errno 17] File exists: 'DataSet_HDFS/HDFS_train.log'

### /!\ : AVANT DE PASSER A LA SUITE, IL FAUT MAINTENANT EXECUTER LE CODE `project_parser.py` 

## Création des Fichiers Structurés de Sortie (après parsing)

*Génération des Fichiers de Logs Structurés*

Une fois le parsing terminé avec `Drain`, nous obtenons deux fichiers :
- `HDFS_2k_train.log_structured.csv` : les logs structurés de l’ensemble d'entraînement.
- `HDFS_2k.log_structured.csv` : les logs structurés de tout le fichier `HDFS.log`.

Nous allons maintenant extraire les logs de test de `HDFS_2k.log_structured.csv` pour créer `HDFS_2k_test.log_structured.csv`.


In [14]:
all_parsed = pd.read_csv('DataSet_HDFS/HDFS.log_structured.csv')

In [15]:
test_parsed = all_parsed.iloc[train_idx:]

In [16]:
test_parsed.to_csv('DataSet_HDFS/HDFS_test.log_structured.csv', index = False)